<div>
<div style="text-align:center; display:block; float:left; padding:80px;"><img width="200px"  src="https://kaggle2.blob.core.windows.net/competitions/kaggle/4651/logos/front_page.png"/><span style="color:red;">**New User Booking**</span></div>
<div style="">
**Objective:** In this recruiting competition, Airbnb challenges you to predict in which country a new user will make his or her first booking.  
  
** Description: ** In this challenge, you are given a list of users along with their demographics, web session records, and some summary statistics. You are asked to predict which country a new user's first booking destination will be. All the users in this dataset are from the USA.
</div>
<img src="https://kaggle2.blob.core.windows.net/competitions/kaggle/4651/media/airbnb_banner.png" />

## 1. Import Packages

In [1]:
using DataFrames
using MLBase

  likely near /Users/diego/.julia/v0.4/MLBase/src/modeltune.jl:5
  likely near /Users/diego/.julia/v0.4/MLBase/src/modeltune.jl:5
  likely near /Users/diego/.julia/v0.4/MLBase/src/modeltune.jl:5
  likely near /Users/diego/.julia/v0.4/MLBase/src/deprecated/datapre.jl:104
  likely near /Users/diego/.julia/v0.4/MLBase/src/deprecated/datapre.jl:105
  likely near /Users/diego/.julia/v0.4/MLBase/src/deprecated/datapre.jl:163
  likely near /Users/diego/.julia/v0.4/MLBase/src/deprecated/datapre.jl:163
  likely near /Users/diego/.julia/v0.4/MLBase/src/deprecated/datapre.jl:163


## 2. Load Data

In [34]:
train = readtable("data/train_users_2.csv.gz")
test = readtable("data/test_users.csv.gz")
people_demograph = readtable("data/age_gender_bkts.csv.gz")
countries_demograph = readtable("data/countries.csv.gz")
user_sessions = readtable("data/sessions.csv.gz")
full = vcat(train, test);

## 3. Data Explorary Analysis I: Initial Exploration

In [3]:
showcols(full);

275547x16 DataFrames.DataFrame
| Col #

In [4]:
head(full)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
1,gxn3p5htnn,2010-06-28,20090319043255,NA,-unknown-,NA,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
2,820tgsjxq7,2011-05-25,20090523174809,NA,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
3,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
4,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
5,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
6,osr2jwljor,2010-01-01,20100101215619,2010-01-02,-unknown-,NA,basic,0,en,other,other,omg,Web,Mac Desktop,Chrome,US


 | Name                    | Eltype     | Missing |
|-------|-------------------------|------------|---------|
| 1     | id                      | UTF8String | 0       |
| 2     | date_account_created    | UTF8String | 0       |
| 3     | timestamp_first_active  | Int64      | 0       |
| 4     | date_first_booking      | Any        | 186639  |
| 5     | gender                  | UTF8String | 0       |
| 6     | age                     | Float64    | 116866  |
| 7     | signup_method           | UTF8String | 0       |
| 8     | signup_flow             | Int64      | 0       |
| 9     | language                | UTF8String | 0       |
| 10    | affiliate_channel       | UTF8String | 0       |
| 11    | affiliate_provider      | UTF8String | 0       |
| 12    | first_affiliate_tracked | UTF8String | 6085    |
| 13    | signup_app              | UTF8String | 0       |
| 14    | first_device_type       | UTF8String | 0       |
| 15    | first_browser           | UTF8String | 0       |
| 16

### 3.1. Variables Categorization

In [5]:
output_var   = [:country_destination]

numvar_list  = [:age]
datevar_list = [:date_account_created, :timestamp_first_active]
catvar_list  = [:gender, :signup_method, :signup_flow, :language, :affiliate_channel, :affiliate_provider,
                :first_affiliate_tracked, :signup_app, :first_device_type, :first_browser];

### 3.2. Handling Missing Values

#### Collect Most Frequent Values and Define as Default

In [6]:
default_age_value = median(dropna(full[:age]))
deafult_first_affiliate_tracked = mode(dropna(full[:first_affiliate_tracked]));

In [7]:
apply_default!(df, column, default_value) = df[isna(df[column]), column] = default_value

apply_default! (generic function with 1 method)

In [8]:
apply_default!(train, :age, default_age_value)
apply_default!(test, :age, default_age_value)
apply_default!(full, :age, default_age_value)

train[train[:age] .> 100, :age] = default_age_value
train[train[:age] .< 18, :age]  = default_age_value

test[test[:age] .> 100, :age]   = default_age_value
test[test[:age] .< 18, :age]    = default_age_value

full[full[:age] .> 100, :age]   = default_age_value
full[full[:age] .< 18, :age]    = default_age_value

apply_default!(train, :first_affiliate_tracked, deafult_first_affiliate_tracked)
apply_default!(test, :first_affiliate_tracked, deafult_first_affiliate_tracked)
apply_default!(full, :first_affiliate_tracked, deafult_first_affiliate_tracked);

In [9]:
showcols(full)

275547x16 DataFrames.DataFrame


## 4. Feature Extraction I: Initial Extraction

### 4.0. Auxiliary Methods

In [10]:
# Feature Extraction
function generate_feature_extraction(df, column, extraction_function)
    array = Array[extraction_function(df[i, column]) for i=1:size(df, 1)]
    nrows, ncols = size(array, 1), size(array[1], 1)
    return reshape(vcat(array'...), nrows, ncols)
end

function arrays_to_matrix(array)
    nrows, ncols = size(array, 1), size(array[1], 1)
    return reshape(vcat(array'...), nrows, ncols)
end

function apply_feature_extraction!(df, feature_names, feature_suffix, feature_matrix)
    for i=1:length(feature_names)
        df[symbol("$(feature_names[i])_$feature_suffix")] = feature_matrix[:, i]
    end
end

# Feature Encoders
generate_feature_encoders(df, columns) = Dict([ column => labelmap(dropna(df[column])) for column in columns ])
apply_feature_encoder!(df, column, encoders) = df[column] = labelencode(encoders[column], df[column])
function apply_feature_encoders!(df, feature_list, encoders)
    for column in feature_list
        apply_feature_encoder!(df, column, encoders)
    end
end

# Binarize Features
function binarize_feature!(df, column, encoders)
    !haskey(encoders, column) && error("Feature $column was not encoded.")
    
    nrows, ncols = size(df, 1), length(keys(encoders[column]))
    feature_matrix = fill(0, nrows, ncols)
    for i=1:nrows
        feature_matrix[i, df[i, column]] = 1
    end
    
    feature_names = [ symbol("$column=$value") for value in keys(encoders[column]) ]
    for i=1:length(feature_names)
        df[feature_names[i]] = feature_matrix[:, i]
    end
end

function binarize_features!(df, feature_list, encoders)
    for column in feature_list
        binarize_feature!(df, column, encoders)
    end
end

binarize_features! (generic function with 1 method)

### 4.1. Date Account Created

In [11]:
date_features_names = [:is_sunday, :is_monday, :is_tuesday, :is_wednesday, :is_thursday, 
                       :is_friday, :is_saturday, :is_weekday, :is_weekend, :is_middle_week,
                       :is_january, :is_february, :is_march, :is_april, :is_may, :is_june,
                       :is_july, :is_august, :is_september, :is_october, :is_november, 
                       :is_december, :is_first_quarter_year, :is_second_quarter_year, 
                       :is_third_quarter_year, :is_forth_quarter_year, :is_first_trimester_year, 
                       :is_second_trimester_year, :is_third_trimester_year, :is_first_half_year, 
                       :is_second_half_year]

function extract_date_features(string_date)
    
    date = Date(string_date)
    
    dayofweek       = Dates.dayofweek(date)    
    is_sunday       = Dates.Sunday == dayofweek? 1 : 0
    is_monday       = Dates.Monday == dayofweek? 1 : 0
    is_tuesday      = Dates.Tuesday == dayofweek? 1 : 0
    is_wednesday    = Dates.Wednesday == dayofweek? 1 : 0
    is_thursday     = Dates.Thursday == dayofweek? 1 : 0
    is_friday       = Dates.Friday == dayofweek? 1 : 0
    is_saturday     = Dates.Saturday == dayofweek? 1 : 0
    is_weekday      = Dates.Monday <= dayofweek <= Dates.Friday? 1 : 0
    is_weekend      = Dates.Saturday <= dayofweek <= Dates.Sunday? 1 : 0
    is_middle_week  = Dates.Tuesday <= dayofweek <= Dates.Thursday? 1 : 0
    
    month = Dates.month(date)
    is_january               = Dates.January == month? 1 : 0
    is_february              = Dates.February == month? 1 : 0
    is_march                 = Dates.March == month? 1 : 0
    is_april                 = Dates.April == month? 1 : 0
    is_may                   = Dates.May == month? 1 : 0
    is_june                  = Dates.June == month? 1 : 0
    is_july                  = Dates.July == month? 1 : 0
    is_august                = Dates.August == month? 1 : 0
    is_september             = Dates.September == month? 1 : 0
    is_october               = Dates.October == month? 1 : 0
    is_november              = Dates.November == month? 1 : 0
    is_december              = Dates.December == month? 1 : 0
    is_first_quarter_year    = month < 4? 1 : 0
    is_second_quarter_year   = 4 <= month < 7? 1 : 0
    is_third_quarter_year    = 7 <= month < 10? 1 : 0
    is_forth_quarter_year    = month >= 10? 1 : 0
    is_first_trimester_year  = month <= 4? 1 : 0
    is_second_trimester_year = 4 < month <= 8? 1 : 0
    is_third_trimester_year  = month > 8? 1 : 0
    is_first_half_year       = month <= 6? 1 : 0
    is_second_half_year      = month > 6? 1 : 0
    
    return [is_sunday, is_monday, is_tuesday, is_wednesday, is_thursday, 
            is_friday, is_saturday, is_weekday, is_weekend, is_middle_week,
            is_january, is_february, is_march, is_april, is_may, is_june,
            is_july, is_august, is_september, is_october, is_november, 
            is_december, is_first_quarter_year, is_second_quarter_year, 
            is_third_quarter_year, is_forth_quarter_year, is_first_trimester_year, 
            is_second_trimester_year, is_third_trimester_year, is_first_half_year, 
            is_second_half_year]
end

extract_date_features (generic function with 1 method)

In [12]:
function extract_date_account_created_features!(df)
    var = :date_account_created
    feature_matrix = generate_feature_extraction(df, var, extract_date_features)
    apply_feature_extraction!(df, date_features_names, var, feature_matrix)
    
    return df
end

extract_date_account_created_features! (generic function with 1 method)

In [13]:
extract_date_account_created_features!(train)
extract_date_account_created_features!(test);

### 4.2. Timestamp First Active

In [14]:
hour_feature_names = [:is_morning, :is_afternoon, :is_evenning, :is_night, :is_late_night, 
                      :is_midday, :is_launch_time, :is_end_work_day]

function extract_time_features(string_time_hhmmss)
    hour = parse(Int, string_time_hhmmss[1:2])
    is_morning = 6 <= hour < 12? 1 : 0
    is_afternoon = 13 <= hour < 18? 1 : 0
    is_evenning = 18 <= hour < 20? 1 : 0
    is_night = 20 <= hour <= 22? 1 : 0
    is_late_night = 22 < hour <= 24 || 0 <= hour < 6? 1: 0
    is_midday = 12 <= hour < 13? 1 : 0
    is_launch_time = 11 <= hour <= 14? 1 : 0
    is_end_work_day = 17 <= hour <= 19? 1 : 0
    
    return [is_morning, is_afternoon, is_evenning, is_night, is_late_night, 
            is_midday, is_launch_time, is_end_work_day]
end

function split_timestamp_in_date_and_time!(df, original_column, column_date_name, column_time_name)
    data_format = Dates.DateFormat("yyyymmddHHMMSS")
    df[column_date_name] = [string(DateTime(string(df[i, original_column]), data_format))[1:10] for i=1:size(df, 1)]
    df[column_time_name] = [string(DateTime(string(df[i, original_column]), data_format))[12:end] for i=1:size(df, 1)]
    return df
end

| Col # | Name                    | Eltype     | Missing |
|-------|-------------------------|------------|---------|
| 1     | id                      | UTF8String | 0       |
| 2     | date_account_created    | UTF8String | 0       |
| 3     | timestamp_first_active  | Int64      | 0       |
| 4     | date_first_booking      | Any        | 186639  |
| 5     | gender                  | UTF8String | 0       |
| 6     | age                     | Float64    | 0       |
| 7     | signup_method           | UTF8String | 0       |
| 8     | signup_flow             | Int64      | 0       |
| 9     | language                | UTF8String | 0       |
| 10    | affiliate_channel       | UTF8String | 0       |
| 11    | affiliate_provider      | UTF8String | 0       |
| 12    | first_affiliate_tracked | UTF8String | 0       |
| 13    | signup_app              | UTF8String | 0       |
| 14    | first_device_type       | UTF8String | 0       |
| 15    | first_browser           | UTF8String | 0      

split_timestamp_in_date_and_time! (generic function with 1 method)

In [15]:
function extract_timestamp_first_active_features!(df)
    var = :timestamp_first_active
    var_1 = :timestamp_first_active_date
    var_2 = :timestamp_first_active_time
    
    split_timestamp_in_date_and_time!(df, var, var_1, var_2)

    feature_matrix = generate_feature_extraction(df, var_1, extract_date_features)
    apply_feature_extraction!(df, date_features_names, var_1, feature_matrix)

    feature_matrix = generate_feature_extraction(df, var_2, extract_time_features)
    apply_feature_extraction!(df, hour_feature_names, var_2, feature_matrix)
    
    return df
end

extract_timestamp_first_active_features! (generic function with 1 method)

In [16]:
extract_timestamp_first_active_features!(train)
extract_timestamp_first_active_features!(test);

### 4.3. Age

In [17]:
age_feature_names = [:is_age_under_25, :is_age_between_25_34, :is_age_between_35_44, 
                     :is_age_between_45_54, :is_age_between_55_64, :is_age_above_64,
                     :is_young_adult, :is_old_adult]

function extract_age_features(age)
    
    is_age_under_25      = age < 25? 1 : 0
    is_age_between_25_34 = 25 <= age < 35? 1 : 0
    is_age_between_35_44 = 35 <= age < 45? 1 : 0
    is_age_between_45_54 = 45 <= age < 55? 1 : 0
    is_age_between_55_64 = 55 <= age < 65? 1 : 0
    is_age_above_64      = age >= 65? 1 : 0
    
    is_young_adult       = 25 <= age < 45? 1 : 0
    is_old_adult         = 45 <= age < 65? 1 : 0
    
    return [is_age_under_25, is_age_between_25_34, is_age_between_35_44, 
            is_age_between_45_54, is_age_between_55_64, is_age_above_64,
            is_young_adult, is_old_adult]
end

extract_age_features (generic function with 1 method)

In [18]:
function extract_age_features!(df)
    
    feature_matrix = generate_feature_extraction(df, :age, extract_age_features)
    apply_feature_extraction!(df, age_feature_names, :age, feature_matrix)
    
    return df
end

extract_age_features! (generic function with 1 method)

In [19]:
extract_age_features!(train)
extract_age_features!(test);

### 4.4. Binarize Categorical Features

In [20]:
encoders = generate_feature_encoders(full, vcat(catvar_list, output_var));

In [21]:
apply_feature_encoders!(train, vcat(catvar_list, output_var), encoders)
apply_feature_encoders!(test, catvar_list, encoders);

In [22]:
binarize_features!(train, catvar_list, encoders)
binarize_features!(test, catvar_list, encoders);

### 4.5. People Demographic Features

In [23]:
function age_bucket2age(age_bucket)
    if age_bucket == "100+" || startswith(age_bucket, "9") || startswith(age_bucket, "8") ||
       startswith(age_bucket, "7") || startswith(age_bucket, "65")
        return 5
    elseif startswith(age_bucket, "60") || startswith(age_bucket, "55")
        return 4
    elseif startswith(age_bucket, "50") || startswith(age_bucket, "45")
        return 3
    elseif startswith(age_bucket, "40") || startswith(age_bucket, "35")
        return 2
    elseif startswith(age_bucket, "30") || startswith(age_bucket, "25")
        return 1
    else
        return 0
    end
end

function age2encoded_age(age)
    age < 25 && return 0
    25 <= age < 35 && return 1
    35 <= age < 45 && return 2
    45 <= age < 55 && return 3
    55 <= age < 65 && return 4
    age >= 65 && return 5
end
    
function get_people_statistics(people_demograph)
    people_demograph[:age] = [string(age_bucket2age(age)) for age in people_demograph[:age_bucket]]
    data = Dict()
    
    for i=1:size(people_demograph, 1)
        key = string(people_demograph[i, :age], people_demograph[i, :gender])
        country = people_demograph[i, :country_destination]
        population = people_demograph[i, :population_in_thousands] 
        
        !haskey(data, key) && (data[key] = Dict())
        !haskey(data[key], country) && (data[key][country] = 0)
        data[key][country] += population
    end
    
    for key in keys(data)
        total_value = sum(values(data[key]))
        for subkey in keys(data[key])
            data[key][subkey] /= total_value
        end
    end
    
    delete!(people_demograph, :age)
    
    return data
end

function add_people_statistics_features!(df, people_stats)
    gender_values = keys(encoders[:gender])
    N_FEATURES = 80
    features = fill(0.0, size(df, 1), N_FEATURES)
    for i=1:size(df, 1)
        age = string(age2encoded_age(df[i, :age]))
        gender = lowercase(string(gender_values[df[i, :gender]]))
        if haskey(people_stats, string(age, gender))
            countries_population = collect(values(people_stats[string(age, gender)]))
            append!(countries_population, [Int(v > .10) for v in countries_population])
            append!(countries_population, [Int(v > .15) for v in countries_population])
            append!(countries_population, [Int(v > .20) for v in countries_population])
            features[i, :] = countries_population
        else
            features[i, :] = fill(0, N_FEATURES)
        end
    end
    
    feature_names = [symbol(:pstats, "_", i) for i=1:N_FEATURES]
    apply_feature_extraction!(df, feature_names, "", features)
    
    return df
end

add_people_statistics_features! (generic function with 1 method)

In [24]:
people_stats = get_people_statistics(people_demograph);

In [25]:
add_people_statistics_features!(train, people_stats)
add_people_statistics_features!(test, people_stats);

### 4.6. Add Top Countries Statistics

In [26]:
country_distance = Dict([countries_demograph[i, :country_destination] => countries_demograph[i, :distance_km] for i=1:size(countries_demograph, 1)])
country_language = Dict([countries_demograph[i, :country_destination] => countries_demograph[i, :destination_language][1:2] for i=1:size(countries_demograph, 1)]);
country_language_levenshtein_distance = Dict([countries_demograph[i, :country_destination] => countries_demograph[i, :language_levenshtein_distance] for i=1:size(countries_demograph, 1)]);
country_language["ES"] = "es"
country_language["PT"] = "pt";

In [27]:
function add_top5_country_stats!(df, people_stats, country_distance, country_language)
    gender_values = keys(encoders[:gender])
    countries_values = keys(encoders[:country_destination])
    language_values = keys(encoders[:language])
    N_FEATURES = 15
    N_TOP = 5
    features = fill(0.0, size(df, 1), N_FEATURES)
    for i=1:size(df, 1)
        age = string(age2encoded_age(df[i, :age]))
        gender = lowercase(string(gender_values[df[i, :gender]]))
        
        !haskey(people_stats, age*gender) && continue
        
        sorted_countries = sort(collect(people_stats[age*gender]), by= x-> last(x), rev=true)
        top_countries = [first(c) for c in sorted_countries[1:N_TOP]]
        top_distances = [country_distance[c] for c in top_countries]
        top_language_levenshtein_distance = [country_language_levenshtein_distance[c] for c in top_countries]
        top_languages = [find(country_language[c] .== language_values)[1] for c in top_countries]
        top_encoded_countries = [find(c .== countries_values)[1] for c in top_countries]
        features[i, :] =  vcat(top_encoded_countries, top_distances, top_languages)
    end
    
    feature_names = [symbol(:cstats, "_", i) for i=1:N_FEATURES]
    apply_feature_extraction!(df, feature_names, "", features)
    
    return df
end

add_top5_country_stats! (generic function with 1 method)

In [28]:
add_top5_country_stats!(train, people_stats, country_distance, country_language)
add_top5_country_stats!(test, people_stats, country_distance, country_language);

### 4.7. Add Sessions Statistics

In [ ]:
user_actions = Set(dropna(user_sessions[:action]))
user_action_types = Set(dropna(user_sessions[:action_type]))
user_action_details = Set(dropna(user_sessions[:action_detail]))

### 4.. Delete Unused Features

In [31]:
features_to_remove_list = vcat([:date_first_booking, :date_account_created, :timestamp_first_active, 
                                :timestamp_first_active_date, :timestamp_first_active_time], 
                                catvar_list)

function delete_unused_features!(df, feature_list)
    for column in feature_list
        haskey(df, column) && delete!(df, column)
    end
end

delete_unused_features! (generic function with 1 method)

In [32]:
delete_unused_features!(train, features_to_remove_list)
delete_unused_features!(test, features_to_remove_list);

## 5. Export Remodeled Dataset

In [33]:
writetable("data/train_v2.tsv", train, separator='\t')
writetable("data/test_v2.tsv", test, separator='\t')
writetable("data/full_v1.tsv", vcat(train, test), separator='\t');